In [ ]:
import numpy as np
import pickle
import re
import os
import sys
if '..' not in sys.path:
    sys.path.append('..')
from pfcommon import parse_Amat_file

In [ ]:
folder = 'default_const_P_loads'
# folder = 'G02_double_H_const_P_loads'
data_folder = os.path.join('..','..','modal_analysis','IEEE39',folder)
vars_file = os.path.join(data_folder, 'VariableToIdx_Jacobian.txt')
var_idx = {}
state_vars, voltages, currents, signals = {}, {}, {}, {}
with open(vars_file, 'r') as fid:
    for line in fid:
        line = line.strip()
        if len(line) == 0:
            continue
        elif ';' in line:
            tokens = [token.lstrip() for token in line.split(';')]
            var_type = tokens[2].lower()
        else:
            idx = int(re.findall('\d+', line)[0]) - 1
            var_name = re.findall('"[a-zA-Z0-9:]*"', line)[0][1:-1]
            obj_name = re.findall('[ ]+.*[ ]+', line)[0].strip().split(os.path.sep)[-1].split('.')[0].replace(' ', '_')
            if obj_name not in var_idx:
                var_idx[obj_name] = {}
                state_vars[obj_name] = []
                voltages[obj_name] = []
                currents[obj_name] = []
                signals[obj_name] = []
            var_idx[obj_name][var_name] = idx
            if 'state' in var_type:
                state_vars[obj_name].append(var_name)
            elif 'voltage' in var_type:
                voltages[obj_name].append(var_name)
            elif 'current' in var_type:
                currents[obj_name].append(var_name)
            elif 'signal' in var_type:
                signals[obj_name].append(var_name)

In [ ]:
J = parse_Amat_file(os.path.join(data_folder, 'Jacobian.mtl'))
A = parse_Amat_file(os.path.join(data_folder, 'Amat.mtl'))
N_state_vars = np.sum([len(v) for v in state_vars.values()])
N_algebraic_vars = J.shape[0] - N_state_vars
print(f'# of state variables: {N_state_vars}.')
print(f'# of algebraic variables: {N_algebraic_vars}.')

In [ ]:
Jfx = J[:N_state_vars, :N_state_vars]
Jfy = J[:N_state_vars, N_state_vars:]
Jgx = J[N_state_vars:, :N_state_vars]
Jgy = J[N_state_vars:, N_state_vars:]
tmp = Jfx - Jfy @ np.linalg.inv(Jgy) @ Jgx
assert np.all(np.abs(A-tmp) < 1e-8)

In [ ]:
col_idx = [var_idx['Bus_03']['ur'], var_idx['Bus_03']['ui']]
if 'Load_03' in var_idx:
    col_idx.append(var_idx['Load_03']['ir:bus1'])
    col_idx.append(var_idx['Load_03']['ii:bus1'])
print(col_idx)
col_idx = np.array(col_idx)

In [ ]:
for i,row in enumerate(J[:,col_idx]):
    if np.any(row != 0):
        sys.stdout.write('{:3d} [ '.format(i))
        for val in row:
            sys.stdout.write('{:8.1f} '.format(val))
        print(']')